In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rajna\\OneDrive\\Desktop\\DLP001\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rajna\\OneDrive\\Desktop\\DLP001'

In [26]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/rishabh6900/DLP001.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="rishabh6900"
os.environ["MLFLOW_TRACKING_PASSWORD"]="31ed2090efbb3357e9e4df34a19443e3622d535f"

In [7]:
# import dagshub
# dagshub.init(repo_owner='rishabh6900', repo_name='DLP001', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

In [27]:

import tensorflow as tf

In [28]:

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [30]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [31]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [32]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Brain-Tumor",
            mlflow_uri="https://dagshub.com/rishabh6900/DLP001.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [33]:

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [36]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [37]:

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    
    # Ensure the MLflow tracking URI is set correctly
    mlflow.set_tracking_uri(eval_config.mlflow_uri)
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-02-23 11:22:55,796: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-23 11:22:55,801: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-23 11:22:55,803: INFO: common: created directory at: artifacts]
Found 1536 images belonging to 2 classes.
96/96 [==============================] - 212s 2s/step - loss: 0.1666 - accuracy: 0.9382
[2025-02-23 11:26:28,650: INFO: common: json file saved at: scores.json]
[2025-02-23 11:26:28,705: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /rishabh6900/DLP001.mlflow/api/2.0/mlflow/runs/create]


2025/02/23 11:26:31 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-02-23 11:26:33,475: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2025-02-23 11:26:34,212: INFO: builder_impl: Assets written to: C:\Users\rajna\AppData\Local\Temp\tmps6itxxgo\model\data\model\assets]


c:\Users\rajna\anaconda3\envs\BT\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/02/23 11:27:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
